**DATAFRAMES & SparkSQL** 

In this notebook, we present our ability to work with PySpark DataFrames and Spark SQL, demonstrating essential data engineering and analytical techniques such as:
- Reading and inspecting DataFrames
- Handling schemas and missing values
- Transformations and filtering
- Aggregations and SQL queries
- Joining tables
- Correlation analysis
- Comparing DataFrame API vs Spark SQL
- Working with multiple datasets (Wine + Heart Failure)

In [1]:
pip install pyspark findspark


[notice] A new release of pip is available: 25.0 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# We begin by installing dependencies and initializing a SparkSession, the entry point to all Spark functionality

In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("MySparkApp") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()

spark

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/11 14:29:15 WARN Utils: Your hostname, Ninas-macbook.local, resolves to a loopback address: 127.0.0.1; using 192.168.2.16 instead (on interface en0)
25/12/11 14:29:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/11 14:29:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Wine Dataset

In [3]:
red_df = spark.read.csv("winequality-red.csv", header=True, inferSchema=True, sep=';')
white_df = spark.read.csv("winequality-white.csv", header=True, inferSchema=True, sep=';')

In [4]:
red_df.show(5)
white_df.show(5)
red_df.printSchema()

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|          7.4|             0.7|        0.0|           1.9|    0.076|               11.0|                34.0| 0.9978|3.51|     0.56|    9.4|      5|
|          7.8|            0.88|        0.0|           2.6|    0.098|               25.0|                67.0| 0.9968| 3.2|     0.68|    9.8|      5|
|          7.8|            0.76|       0.04|           2.3|    0.092|               15.0|                54.0|  0.997|3.26|     0.65|    9.8|      5|
|         11.2|            0.28|       0.56|           1.9|    0.075|               17.0|           

# Adding Columns & Combining Datasets

In [5]:
from pyspark.sql.functions import lit

red_df = red_df.withColumn("wine_type", lit("red"))
white_df = white_df.withColumn("wine_type", lit("white"))

In [6]:
wine_df = red_df.union(white_df)
wine_df.show(5)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+---------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|wine_type|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+---------+
|          7.4|             0.7|        0.0|           1.9|    0.076|               11.0|                34.0| 0.9978|3.51|     0.56|    9.4|      5|      red|
|          7.8|            0.88|        0.0|           2.6|    0.098|               25.0|                67.0| 0.9968| 3.2|     0.68|    9.8|      5|      red|
|          7.8|            0.76|       0.04|           2.3|    0.092|               15.0|                54.0|  0.997|3.26|     0.65|    9.8|      5|      red|
|         11.2|            0.28|       0

# Basic DataFrame operations:

## 1. Counting rows:

In [7]:
wine_df.count()

6497

## 2. Summary statistics

In [8]:
wine_df.describe().show()

25/12/11 14:29:20 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-----------------+-------------------+-------------------+------------------+-------------------+-------------------+--------------------+--------------------+-------------------+-------------------+------------------+------------------+---------+
|summary|    fixed acidity|   volatile acidity|        citric acid|    residual sugar|          chlorides|free sulfur dioxide|total sulfur dioxide|             density|                 pH|          sulphates|           alcohol|           quality|wine_type|
+-------+-----------------+-------------------+-------------------+------------------+-------------------+-------------------+--------------------+--------------------+-------------------+-------------------+------------------+------------------+---------+
|  count|             6497|               6497|               6497|              6497|               6497|               6497|                6497|                6497|               6497|               6497|              6497|  

## 3. Checking for Null values

In [9]:
from pyspark.sql.functions import col, when, count, isnan

numeric_cols = [c for c, t in wine_df.dtypes if t in ("double", "int", "float")]
string_cols = [c for c, t in wine_df.dtypes if t not in ("double", "int", "float")]

missing = wine_df.select(
    # numeric columns — check NULL or NaN
    *[
        count(when(col(c).isNull() | isnan(c), c)).alias(c)
        for c in numeric_cols
    ],
    # string columns — only check NULL
    *[
        count(when(col(c).isNull(), c)).alias(c)
        for c in string_cols
    ]
)

missing.show()


+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+---+---------+-------+-------+---------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density| pH|sulphates|alcohol|quality|wine_type|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+---+---------+-------+-------+---------+
|            0|               0|          0|             0|        0|                  0|                   0|      0|  0|        0|      0|      0|        0|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+---+---------+-------+-------+---------+



# SparkSQL Queries

In [10]:
wine_df.createOrReplaceTempView("wines")

## 1. Average alcohol content by wine type

In [11]:
spark.sql("""
    SELECT wine_type, AVG(alcohol) AS avg_alcohol
    FROM wines
    GROUP BY wine_type
""").show()


+---------+------------------+
|wine_type|       avg_alcohol|
+---------+------------------+
|      red|10.422983114446502|
|    white|10.514267047774638|
+---------+------------------+



## 2. Count wines by quality rating

In [12]:
spark.sql("""
    SELECT quality, COUNT(*) AS count
    FROM wines
    GROUP BY quality
    ORDER BY quality
""").show()

+-------+-----+
|quality|count|
+-------+-----+
|      3|   30|
|      4|  216|
|      5| 2138|
|      6| 2836|
|      7| 1079|
|      8|  193|
|      9|    5|
+-------+-----+



## 3. Highest alcohol red wines

In [13]:
spark.sql("""
    SELECT *
    FROM wines
    WHERE wine_type = 'red'
    ORDER BY alcohol DESC
    LIMIT 5
""").show()

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+---------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|wine_type|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+---------+
|         15.9|            0.36|       0.65|           7.5|    0.096|               22.0|                71.0| 0.9976|2.98|     0.84|   14.9|      5|      red|
|          5.2|            0.34|        0.0|           1.8|     0.05|               27.0|                63.0| 0.9916|3.68|     0.79|   14.0|      6|      red|
|          5.0|            0.42|       0.24|           2.0|     0.06|               19.0|                50.0| 0.9917|3.72|     0.74|   14.0|      8|      red|
|          4.9|            0.42|        

## 4. Correlation: alcohol vs quality

In [14]:
spark.sql("""
    SELECT corr(alcohol, quality) AS alcohol_quality_corr
    FROM wines;
""").show()

+--------------------+
|alcohol_quality_corr|
+--------------------+
|  0.4443185200076535|
+--------------------+



# Deeper analysis

# 1. *Which chemical features differentiate red vs white wines the most?*

In [15]:
spark.sql("""
    SELECT 
        wine_type,
        AVG(alcohol) AS avg_alcohol,
        AVG(`residual sugar`) AS avg_sugar,
        AVG(`volatile acidity`) AS avg_vol_acid,
        AVG(sulphates) AS avg_sulphates
    FROM wines
    GROUP BY wine_type
    ORDER BY wine_type
""").show()


+---------+------------------+------------------+-------------------+------------------+
|wine_type|       avg_alcohol|         avg_sugar|       avg_vol_acid|     avg_sulphates|
+---------+------------------+------------------+-------------------+------------------+
|      red|10.422983114446502|2.5388055034396517| 0.5278205128205131|0.6581488430268921|
|    white|10.514267047774638| 6.391414863209486|0.27824111882401087|0.4898468762760325|
+---------+------------------+------------------+-------------------+------------------+



White wines have much higher residual sugar.
Red wines have higher volatile acidity and higher sulphates.
Alcohol differences are moderate.

# 2. *Quality distribution by wine type.*


In [16]:
spark.sql("""
    SELECT 
        wine_type,
        quality,
        COUNT(*) AS count
    FROM wines
    GROUP BY wine_type, quality
    ORDER BY wine_type, quality
""").show()


+---------+-------+-----+
|wine_type|quality|count|
+---------+-------+-----+
|      red|      3|   10|
|      red|      4|   53|
|      red|      5|  681|
|      red|      6|  638|
|      red|      7|  199|
|      red|      8|   18|
|    white|      3|   20|
|    white|      4|  163|
|    white|      5| 1457|
|    white|      6| 2198|
|    white|      7|  880|
|    white|      8|  175|
|    white|      9|    5|
+---------+-------+-----+



White wines dominate the higher quality categories (7–9).
Red wines have more samples around quality 5–6

# 3. *Correlations between multiple features and quality (pairwise)*

In [17]:
spark.sql("""
    SELECT 
        corr(quality, alcohol) AS corr_quality_alcohol,
        corr(quality, sulphates) AS corr_quality_sulphates,
        corr(quality, `volatile acidity`) AS corr_quality_volacid,
        corr(quality, `residual sugar`) AS corr_quality_sugar
    FROM wines
""").show()

+--------------------+----------------------+--------------------+--------------------+
|corr_quality_alcohol|corr_quality_sulphates|corr_quality_volacid|  corr_quality_sugar|
+--------------------+----------------------+--------------------+--------------------+
| 0.44431852000765354|  0.038485445876513875| -0.2656994776114671|-0.03698048458576981|
+--------------------+----------------------+--------------------+--------------------+



Quality correlates positively with alcohol and sulphates.
Quality correlates negatively with volatile acidity.
Sugar has almost no positive correlation.

# Heart Failure Dataset
## Load dataset

In [18]:
heart = spark.read.csv("heart.csv", header=True, inferSchema=True, sep=',')
heart.createOrReplaceTempView("heart")
heart.printSchema()
spark.sql("SELECT Age, Cholesterol FROM heart WHERE Age > 50").show()

root
 |-- Age: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- ChestPainType: string (nullable = true)
 |-- RestingBP: integer (nullable = true)
 |-- Cholesterol: integer (nullable = true)
 |-- FastingBS: integer (nullable = true)
 |-- RestingECG: string (nullable = true)
 |-- MaxHR: integer (nullable = true)
 |-- ExerciseAngina: string (nullable = true)
 |-- Oldpeak: double (nullable = true)
 |-- ST_Slope: string (nullable = true)
 |-- HeartDisease: integer (nullable = true)

+---+-----------+
|Age|Cholesterol|
+---+-----------+
| 54|        195|
| 54|        208|
| 58|        164|
| 54|        273|
| 60|        248|
| 53|        260|
| 52|        284|
| 53|        468|
| 51|        188|
| 53|        518|
| 56|        167|
| 54|        224|
| 65|        306|
| 54|        230|
| 54|        294|
| 52|        259|
| 59|        318|
| 52|        180|
| 51|        194|
| 58|        213|
+---+-----------+
only showing top 20 rows


## Filter patients older than 50

In [19]:
#Select with dataframe

heart.select("Age", "Sex", "HeartDisease").show(5)

+---+---+------------+
|Age|Sex|HeartDisease|
+---+---+------------+
| 40|  M|           0|
| 49|  F|           1|
| 37|  M|           0|
| 48|  F|           1|
| 54|  M|           0|
+---+---+------------+
only showing top 5 rows


In [20]:
#Select with Spark SQL
spark.sql("""
    SELECT Age, Sex, HeartDisease
    FROM heart
    LIMIT 5
""").show()

# Both give the same output; SQL is concise for analysts, DSL is good for Python developers.

+---+---+------------+
|Age|Sex|HeartDisease|
+---+---+------------+
| 40|  M|           0|
| 49|  F|           1|
| 37|  M|           0|
| 48|  F|           1|
| 54|  M|           0|
+---+---+------------+



## Filtering patients older than 50

In [21]:
spark.sql("""
    SELECT *
    FROM heart
    WHERE Age > 50
    LIMIT 5
""").show()

+---+---+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+
|Age|Sex|ChestPainType|RestingBP|Cholesterol|FastingBS|RestingECG|MaxHR|ExerciseAngina|Oldpeak|ST_Slope|HeartDisease|
+---+---+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+
| 54|  M|          NAP|      150|        195|        0|    Normal|  122|             N|    0.0|      Up|           0|
| 54|  M|          ATA|      110|        208|        0|    Normal|  142|             N|    0.0|      Up|           0|
| 58|  M|          ATA|      136|        164|        0|        ST|   99|             Y|    2.0|    Flat|           1|
| 54|  F|          ATA|      120|        273|        0|    Normal|  150|             N|    1.5|    Flat|           0|
| 60|  M|          ASY|      100|        248|        0|    Normal|  125|             N|    1.0|    Flat|           1|
+---+---+-------------+---------+-----------+---------+-

## Selecting

In [22]:
# Aggregation with Spark SQL

spark.sql("""
    SELECT Sex, AVG(MaxHR) AS avg_maxhr
    FROM heart
    GROUP BY Sex
""").show()


+---+------------------+
|Sex|         avg_maxhr|
+---+------------------+
|  F|146.13989637305698|
|  M|134.32551724137932|
+---+------------------+



# Grouping & Aggregations

In [23]:
spark.sql("""
 SELECT Sex, AVG(MaxHR) AS avg_maxhr
 FROM heart
 GROUP BY Sex
""").show()

+---+------------------+
|Sex|         avg_maxhr|
+---+------------------+
|  F|146.13989637305698|
|  M|134.32551724137932|
+---+------------------+



Observation: Women have significantly higher average MaxHR than men.

## Self Join Example

In [24]:
# Self-Join: Find pairs of patients who share the same chest pain type with Spark SQL

spark.sql("""
    SELECT a.Age AS Age1, b.Age AS Age2, a.ChestPainType
    FROM heart a
    JOIN heart b ON a.ChestPainType = b.ChestPainType
    LIMIT 5
""").show()

+----+----+-------------+
|Age1|Age2|ChestPainType|
+----+----+-------------+
|  40|  57|          ATA|
|  40|  41|          ATA|
|  40|  55|          ATA|
|  40|  56|          ATA|
|  40|  56|          ATA|
+----+----+-------------+



# Deeper analysis

# 1. *Heart disease prevalence by sex*


In [25]:
spark.sql("""
    SELECT 
    Sex,
    COUNT(*) AS total,
    SUM(HeartDisease) AS num_disease,
    ROUND(SUM(HeartDisease) / COUNT(*), 3) AS disease_rate
    FROM heart
    GROUP BY Sex;
""").show()

+---+-----+-----------+------------+
|Sex|total|num_disease|disease_rate|
+---+-----+-----------+------------+
|  F|  193|         50|       0.259|
|  M|  725|        458|       0.632|
+---+-----+-----------+------------+



Men have significantly higher incidence of heart disease than women

# 2. *Average cholesterol by heart disease status*

In [26]:
spark.sql("""
    SELECT 
    HeartDisease,
    ROUND(AVG(Cholesterol), 2) AS avg_chol
    FROM heart
    GROUP BY HeartDisease;
""").show()

+------------+--------+
|HeartDisease|avg_chol|
+------------+--------+
|           1|  175.94|
|           0|  227.12|
+------------+--------+



Patients with heart disease often have higher cholesterol.

# 3. *Does age affect cholesterol levels*

In [27]:
spark.sql("""
    SELECT 
        CASE 
            WHEN Age < 40 THEN 'Under 40'
            WHEN Age BETWEEN 40 AND 60 THEN '40–60'
            ELSE '60+'
        END AS age_group,
        ROUND(AVG(Cholesterol), 2) AS avg_cholesterol
    FROM heart
    GROUP BY age_group
    ORDER BY avg_cholesterol DESC;
""").show()

+---------+---------------+
|age_group|avg_cholesterol|
+---------+---------------+
|    40–60|         204.61|
| Under 40|         203.48|
|      60+|         180.87|
+---------+---------------+



Cholesterol tends to increase with age group.